# Monte Carlo particle filtering simulation

In [1]:
# initial setup/imports

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import copy

import State

### True-state setup

We need to initialise the system and setup the true state. Here, we can set the evolution, and measurement characteristics.

In [2]:
initPos = np.array([0,0])
initV = np.array([1,0])

trueState = State.Velocity_nD(initPos,initV,False) # false doesn't record the previous state.

### Monte Carlo particle initialisation

In this section, we'll initialise the Monte Carlo particles that will evolve along trajectories defined by their state evolution functions.

In [5]:
n_particles = 1000
particles = []

for i in range(n_particles):
    noise1 = np.random.normal(np.random.normal(0,2,1),1,2)
    noise2 = np.random.normal(np.random.normal(0,2,1),1,2)

    p = State.Velocity_nD(initPos+noise1, initV+noise2, False)
    particles.append(p)

# normalised relative weights. Initially they are equal as no measurements have been made.
weights = np.array([1/n_particles] * n_particles)

In [ ]:
'''Function to resample the Monte Carlo trajectories to enable the prouction of higher-weighted particles

This will need to determine which trajectories to cull (diffreence in log(weights)?) and then sample which trajectories to reproduce based on the cdf of their weights.
'''
def resample(particles,weights):
    cull_value = -4 # factor power of 10 that a weight can be below the maximum weight before it is culled 
    logweights = np.log10(weights)
    cull = (logweights - np.max(logweights)) < cull_value
    n_cull = np.sum(cull)
    cull_indices = np.nonzero(~cull)

    # to get the new trajectories, we need to first simulate a random selection on the valid trajectories to copy, based on their weights
    cdf_weights = np.cumsum(weights*cull)
    rn = np.random.uniform(size=n_cull) * cdf_weights[-1]
    for j,cull_ind in enumerate(cull_indices):
        copy_ind = np.max(np.nonzero(cdf_weights < rn[j]))
        particles[cull_ind] = copy.copy(particles[copy_ind])
        weights[cull_ind] = weights[copy_ind]

    return particles, weights, cull_indices